In [29]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt

os.chdir("/Users/jacobrichards/Desktop/DS_DA_Projects/3-90+_DaysPastDue/data")

train = pd.read_csv("train.csv", na_values=["", "NA"])

test = pd.read_csv("test.csv", na_values=["", "NA"])


import warnings
warnings.filterwarnings('ignore')


In [30]:
lower_bound = train['feature_3'].quantile(0.01)
upper_bound = train['feature_3'].quantile(0.99)

train = train[(train['feature_3'] >= lower_bound) & (train['feature_3'] <= upper_bound)]

In [31]:
train_median = train['feature_3'].median()
train['feature_3'].fillna(train_median, inplace=True)

test_median = test['feature_3'].median()
test['feature_3'].fillna(test_median, inplace=True)

In [32]:
train['date'] = pd.to_datetime(train['date']).dt.year
test['date'] = pd.to_datetime(test['date']).dt.year

def impute_feature_2(df):
    df = df.sort_values(by=['id', 'date'])
    df['feature_2'] = df['feature_2'].fillna(method='ffill')
    df['feature_2'] = df['feature_2'].fillna(method='bfill')
    return df

train = train.groupby('id', group_keys=False).apply(impute_feature_2)
test = test.groupby('id', group_keys=False).apply(impute_feature_2)

In [33]:
train['y'] = train['y'].apply(lambda x: 1 if x == "90+DPD" else 0 if x == "active" else x)
test['y'] = test['y'].apply(lambda x: 1 if x == "90+DPD" else 0 if x == "active" else x)

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train[['feature_1', 'feature_2', 'feature_3', 'feature_4']] = scaler.fit_transform(train[['feature_1', 'feature_2', 'feature_3', 'feature_4']])
test[['feature_1', 'feature_2', 'feature_3', 'feature_4']] = scaler.transform(test[['feature_1', 'feature_2', 'feature_3', 'feature_4']])

In [35]:
train['feature_1_x_feature_2'] = train['feature_1'] * train['feature_2']
test['feature_1_x_feature_2'] = test['feature_1'] * test['feature_2']

train['feature_1_x_feature_3'] = train['feature_1'] * train['feature_3']
test['feature_1_x_feature_3'] = test['feature_1'] * test['feature_3']

train['feature_1_x_feature_4'] = train['feature_1'] * train['feature_4']
test['feature_1_x_feature_4'] = test['feature_1'] * test['feature_4']

train['feature_2_x_feature_3'] = train['feature_2'] * train['feature_3']
test['feature_2_x_feature_3'] = test['feature_2'] * test['feature_3']

train['feature_2_x_feature_4'] = train['feature_2'] * train['feature_4']
test['feature_2_x_feature_4'] = test['feature_2'] * test['feature_4']

train['feature_3_x_feature_4'] = train['feature_3'] * train['feature_4']
test['feature_3_x_feature_4'] = test['feature_3'] * test['feature_4']

In [ ]:
print("\nTrain Data:")
print(f"Total rows: {len(train)}")
print(f"Number of defaults (y=1): {(train['y']==1).sum()}")
print(f"Default rate: {(train['y']==1).mean():.2%}")

print("\nTest Data:")
print(f"Total rows: {len(test)}")
print(f"Number of defaults (y=1): {(test['y']==1).sum()}")
print(f"Default rate: {(test['y']==1).mean():.2%}")

common_ids = set(train['id']) & set(test['id'])
print(f"\nNumber of overlapping IDs between train and test: {len(common_ids)}")
if len(common_ids) > 0:
    print("\nOverlapping IDs:")
    print(common_ids)


In [ ]:
from sklearn.metrics import roc_curve, recall_score, roc_auc_score
from sklearn.preprocessing import SplineTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np

def create_split_splines(data_train, data_test, interaction_term, n_knots=10, degree=3):
    interaction_train = data_train[interaction_term[0]] * data_train[interaction_term[1]]
    interaction_test = data_test[interaction_term[0]] * data_test[interaction_term[1]]
    
    spline_left = SplineTransformer(n_knots=n_knots, degree=degree)
    spline_right = SplineTransformer(n_knots=n_knots, degree=degree)
    
    left_mask_train = interaction_train <= 0
    right_mask_train = ~left_mask_train
    left_mask_test = interaction_test <= 0 
    right_mask_test = ~left_mask_test
    
    X_train_spline = np.zeros((len(data_train), 2 * n_knots + 2 * degree - 2))
    X_test_spline = np.zeros((len(data_test), 2 * n_knots + 2 * degree - 2))
    
    n_features = n_knots + degree - 1
    X_train_spline[left_mask_train, :n_features] = spline_left.fit_transform(interaction_train[left_mask_train].values.reshape(-1, 1))
    X_train_spline[right_mask_train, n_features:] = spline_right.fit_transform(interaction_train[right_mask_train].values.reshape(-1, 1))
    X_test_spline[left_mask_test, :n_features] = spline_left.transform(interaction_test[left_mask_test].values.reshape(-1, 1))
    X_test_spline[right_mask_test, n_features:] = spline_right.transform(interaction_test[right_mask_test].values.reshape(-1, 1))
    
    return X_train_spline, X_test_spline, spline_left, spline_right

features = ['feature_1', 'feature_2', 'feature_3', 'feature_4']
splined_interactions = [['feature_1', 'feature_3'], ['feature_3', 'feature_4']]
regular_interactions = [['feature_1', 'feature_2'], ['feature_2', 'feature_3'], ['feature_2', 'feature_4']]

n_knots = 10
degree = 3

X_train = train[features]
X_test = test[features]
y_train = train['y']
y_test = test['y']

X_train_splines = [X_train]
X_test_splines = [X_test]
spline_transformers = {}

for i, interaction in enumerate(splined_interactions):
    X_train_spline, X_test_spline, spline_left, spline_right = create_split_splines(train, test, interaction)
    X_train_splines.append(X_train_spline)
    X_test_splines.append(X_test_spline)
    spline_transformers[i] = (spline_left, spline_right)

for interaction in regular_interactions:
    X_train_splines.append((train[interaction[0]] * train[interaction[1]]).values.reshape(-1, 1))
    X_test_splines.append((test[interaction[0]] * test[interaction[1]]).values.reshape(-1, 1))

X_train_final = np.hstack(X_train_splines)
X_test_final = np.hstack(X_test_splines)

model = LogisticRegression(fit_intercept=True)
model.fit(X_train_final, y_train)

coefficients = pd.DataFrame(
    {'Feature': features + 
     [f"{interaction[0]}_x_{interaction[1]}_spline_{i}" for interaction in splined_interactions for i in range(2 * n_knots + 2 * degree - 2)] +
     [f"{interaction[0]}_x_{interaction[1]}" for interaction in regular_interactions],
     'Coefficient': model.coef_[0]
    })
print("\nModel Coefficients:")
print(coefficients)

test_pred_proba = model.predict_proba(X_test_final)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, test_pred_proba)
optimal_threshold = thresholds[np.argmin(np.abs(fpr - (1-tpr)))]
y_pred = (test_pred_proba >= optimal_threshold).astype(int)

final_recall = recall_score(y_test, y_pred)
final_auc = roc_auc_score(y_test, test_pred_proba)

plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curve (Test Recall = {final_recall:.3f}, Test AUC = {final_auc:.3f})')
plt.show()

print(f"Optimal threshold: {optimal_threshold:.3f}")
print(f"Final test recall at optimal threshold: {final_recall:.3f}")
print(f"Final test AUC: {final_auc:.3f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

X = train[['feature_1', 'feature_2', 'feature_3', 'feature_4']]
y = train['y']

lr_model = LogisticRegression(random_state=42)
lr_model.fit(X, y)

mean_values = X.mean()
coefficients = lr_model.coef_[0]

def create_plots(plot_type_left, plot_type_right, feature, n_bins):
    plt.figure(figsize=(20, 10))
    
    x_range = np.linspace(X[feature].min(), X[feature].max(), 100)
    pred_data = np.tile(mean_values, (100, 1))
    pred_data = pd.DataFrame(pred_data, columns=X.columns)
    pred_data[feature] = x_range
    y_pred = lr_model.predict_proba(pred_data)[:, 1]
    
    bins = pd.qcut(X[feature], q=n_bins, duplicates='drop')
    bin_means = X.groupby(bins)[feature].mean()
    bin_probs = train.groupby(bins)['y'].mean()
    
    plt.subplot(1, 2, 1)
    plt.plot(x_range, y_pred, label='Predicted Probability', color='red')
    sns.scatterplot(x=bin_means, y=bin_probs, color='blue', s=20, label='Observed Probability')
    plt.text(0.05, 0.95, f'β{list(X.columns).index(feature)+1} = {coefficients[list(X.columns).index(feature)]:.3f}', transform=plt.gca().transAxes)
    plt.title(f'Probabilities vs {feature}\n(Other Variables at Mean)')
    plt.ylabel('Probability')
    plt.xlabel(feature)
    plt.ylim(0, 1)
    plt.xlim(X[feature].min(), X[feature].max())
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(x_range, y_pred, label='Predicted Probability', color='red')
    sns.scatterplot(x=bin_means, y=bin_probs, color='blue', s=20, label='Observed Probability')
    plt.title(f'Probabilities vs {feature}\n(Bins: {n_bins})')
    plt.ylabel('Probability')
    plt.xlabel(feature)
    plt.ylim(0, 1)
    plt.xlim(X[feature].min(), X[feature].max())
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

plot_type_left = widgets.Dropdown(
    options=['Predicted & Observed'],
    description='Left Plot:',
    value='Predicted & Observed'
)

plot_type_right = widgets.Dropdown(
    options=['Predicted & Observed'],
    description='Right Plot:',
    value='Predicted & Observed'
)

feature_selector = widgets.Dropdown(
    options=['feature_1', 'feature_2', 'feature_3', 'feature_4'],
    description='Feature:',
    value='feature_1'
)

bin_slider = widgets.IntSlider(
    value=50,
    min=10,
    max=100,
    step=5,
    description='Bins:'
)

def on_change(change):
    clear_output(wait=True)
    display(plot_type_left)
    display(plot_type_right)
    display(feature_selector)
    display(bin_slider)
    create_plots(plot_type_left.value, plot_type_right.value, feature_selector.value, bin_slider.value)

plot_type_left.observe(on_change, names='value')
plot_type_right.observe(on_change, names='value')
feature_selector.observe(on_change, names='value')
bin_slider.observe(on_change, names='value')

display(plot_type_left)
display(plot_type_right)
display(feature_selector)
display(bin_slider)
create_plots(plot_type_left.value, plot_type_right.value, feature_selector.value, bin_slider.value)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.interpolate import LSQUnivariateSpline

X = train[['feature_1', 'feature_2', 'feature_3', 'feature_4']]
y = train['y']

lr_model = LogisticRegression(random_state=42)
lr_model.fit(X, y)

mean_values = X.mean()
coefficients = lr_model.coef_[0]

def create_plots(feature, n_bins, knot1_val, knot2_val, knot3_val):
    plt.figure(figsize=(10, 6))
    
    bins = pd.qcut(X[feature], q=n_bins, duplicates='drop')
    bin_means = X.groupby(bins)[feature].mean()
    bin_probs = train.groupby(bins)['y'].mean()
    
    x_range = np.linspace(X[feature].min(), X[feature].max(), 100)
    knots = np.array([knot1_val, knot2_val, knot3_val])
    
    spline = LSQUnivariateSpline(bin_means, bin_probs, knots, k=1)
    y_pred = spline(x_range)
    
    plt.plot(x_range, y_pred, label='Piecewise Linear Fit', color='red')
    sns.scatterplot(x=bin_means, y=bin_probs, color='blue', s=20, label='Observed Probability')
    plt.vlines(knots, 0, 1, colors='green', linestyles='dashed', alpha=0.5, label='Knots')
    plt.text(0.05, 0.95, f'β{list(X.columns).index(feature)+1} = {coefficients[list(X.columns).index(feature)]:.3f}', transform=plt.gca().transAxes)
    plt.title(f'Probabilities vs {feature}\n(Other Variables at Mean)')
    plt.ylabel('Probability')
    plt.xlabel(feature)
    plt.ylim(0, 1)
    plt.xlim(X[feature].min(), X[feature].max())
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    print(f"Knot locations: {knots}")
    plt.show()

feature_selector = widgets.Dropdown(
    options=['feature_1', 'feature_2', 'feature_3', 'feature_4'],
    description='Feature:',
    value='feature_1'
)

bin_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=100,
    step=5,
    description='Bins:'
)

knot1 = widgets.FloatSlider(
    value=X[feature_selector.value].min() + (X[feature_selector.value].max()-X[feature_selector.value].min())*0.25,
    min=X[feature_selector.value].min() - 2,
    max=X[feature_selector.value].max() + 2,
    description='Knot 1:'
)

knot2 = widgets.FloatSlider(
    value=X[feature_selector.value].min() + (X[feature_selector.value].max()-X[feature_selector.value].min())*0.5,
    min=X[feature_selector.value].min() - 2,
    max=X[feature_selector.value].max() + 2,
    description='Knot 2:'
)

knot3 = widgets.FloatSlider(
    value=X[feature_selector.value].min() + (X[feature_selector.value].max()-X[feature_selector.value].min())*0.75,
    min=X[feature_selector.value].min() - 2,
    max=X[feature_selector.value].max() + 2,
    description='Knot 3:'
)

def on_change(change):
    clear_output(wait=True)
    display(feature_selector)
    display(bin_slider)
    display(knot1)
    display(knot2)
    display(knot3)
    create_plots(feature_selector.value, bin_slider.value, knot1.value, knot2.value, knot3.value)

feature_selector.observe(on_change, names='value')
bin_slider.observe(on_change, names='value')
knot1.observe(on_change, names='value')
knot2.observe(on_change, names='value')
knot3.observe(on_change, names='value')

display(feature_selector)
display(bin_slider)
display(knot1)
display(knot2)
display(knot3)
create_plots(feature_selector.value, bin_slider.value, knot1.value, knot2.value, knot3.value)

In [ ]:
from sklearn.metrics import roc_curve, recall_score, roc_auc_score, log_loss
from sklearn.preprocessing import SplineTransformer
from sklearn.neural_network import MLPClassifier
import numpy as np

def create_split_splines(data_train, data_test, interaction_term, n_knots=10, degree=3):
    interaction_train = data_train[interaction_term[0]] * data_train[interaction_term[1]]
    interaction_test = data_test[interaction_term[0]] * data_test[interaction_term[1]]
    
    spline_left = SplineTransformer(n_knots=n_knots, degree=degree)
    spline_right = SplineTransformer(n_knots=n_knots, degree=degree)
    
    left_mask_train = interaction_train <= 0
    right_mask_train = ~left_mask_train
    left_mask_test = interaction_test <= 0 
    right_mask_test = ~left_mask_test
    
    X_train_spline = np.zeros((len(data_train), 2 * n_knots + 2 * degree - 2))
    X_test_spline = np.zeros((len(data_test), 2 * n_knots + 2 * degree - 2))
    
    n_features = n_knots + degree - 1
    X_train_spline[left_mask_train, :n_features] = spline_left.fit_transform(interaction_train[left_mask_train].values.reshape(-1, 1))
    X_train_spline[right_mask_train, n_features:] = spline_right.fit_transform(interaction_train[right_mask_train].values.reshape(-1, 1))
    X_test_spline[left_mask_test, :n_features] = spline_left.transform(interaction_test[left_mask_test].values.reshape(-1, 1))
    X_test_spline[right_mask_test, n_features:] = spline_right.transform(interaction_test[right_mask_test].values.reshape(-1, 1))
    
    return X_train_spline, X_test_spline, spline_left, spline_right

features = ['feature_1', 'feature_2', 'feature_3', 'feature_4']
splined_interactions = [['feature_1', 'feature_3'], ['feature_3', 'feature_4']]
regular_interactions = [['feature_1', 'feature_2'], ['feature_2', 'feature_3'], ['feature_2', 'feature_4']]

n_knots = 10
degree = 3

X_train = train[features]
X_test = test[features]
y_train = train['y']
y_test = test['y']

X_train_splines = [X_train]
X_test_splines = [X_test]
spline_transformers = {}

for i, interaction in enumerate(splined_interactions):
    X_train_spline, X_test_spline, spline_left, spline_right = create_split_splines(train, test, interaction)
    X_train_splines.append(X_train_spline)
    X_test_splines.append(X_test_spline)
    spline_transformers[i] = (spline_left, spline_right)

for interaction in regular_interactions:
    X_train_splines.append((train[interaction[0]] * train[interaction[1]]).values.reshape(-1, 1))
    X_test_splines.append((test[interaction[0]] * test[interaction[1]]).values.reshape(-1, 1))

X_train_final = np.hstack(X_train_splines)
X_test_final = np.hstack(X_test_splines)

model = MLPClassifier(hidden_layer_sizes=(128, 128), activation='relu', solver='sgd', learning_rate_init=0.01, random_state=42, warm_start=True)

train_losses = []
test_losses = []
max_epochs = 100

for epoch in range(1, max_epochs + 1):
    model.max_iter = epoch
    model.partial_fit(X_train_final, y_train, classes=np.unique(y_train))
    
    train_pred_proba = model.predict_proba(X_train_final)
    test_pred_proba = model.predict_proba(X_test_final)
    
    train_losses.append(log_loss(y_train, train_pred_proba))
    test_losses.append(log_loss(y_test, test_pred_proba))

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

ax1.plot(range(1, max_epochs + 1), train_losses, label="Training Loss")
ax1.plot(range(1, max_epochs + 1), test_losses, label="Test Loss", linestyle="--")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Loss")
ax1.set_title("Train vs Test Loss Curve")
ax1.legend()
ax1.grid(True)

fpr_train, tpr_train, _ = roc_curve(y_train, train_pred_proba[:, 1])
fpr_test, tpr_test, thresholds = roc_curve(y_test, test_pred_proba[:, 1])

train_auc = roc_auc_score(y_train, train_pred_proba[:, 1])
test_auc = roc_auc_score(y_test, test_pred_proba[:, 1])

ax2.plot(fpr_train, tpr_train, label=f'Train (AUC = {train_auc:.3f})')
ax2.plot(fpr_test, tpr_test, label=f'Test (AUC = {test_auc:.3f})')
ax2.plot([0, 1], [0, 1], 'k--')
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('ROC Curves: Train vs Test')
ax2.legend()

train_scores = []
test_scores = []
thresholds_range = np.linspace(0, 1, 100)

for threshold in thresholds_range:
    train_pred = (train_pred_proba[:, 1] >= threshold).astype(int)
    test_pred = (test_pred_proba[:, 1] >= threshold).astype(int)
    train_scores.append(recall_score(y_train, train_pred))
    test_scores.append(recall_score(y_test, test_pred))

ax3.plot(thresholds_range, train_scores, label='Train')
ax3.plot(thresholds_range, test_scores, label='Test')
ax3.set_xlabel('Threshold')
ax3.set_ylabel('Recall Score')
ax3.set_title('Recall vs Threshold: Train vs Test')
ax3.legend()

plt.tight_layout()
plt.show()

plt.figure(figsize=(6.4, 4.8))
plt.plot(fpr_test, tpr_test, label=f'Test (AUC = {test_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

fnr = 1 - tpr_test
optimal_idx = np.argmin(np.abs(fpr_test - fnr))
optimal_threshold = thresholds[optimal_idx]

y_pred = (test_pred_proba[:, 1] >= optimal_threshold).astype(int)
recall = recall_score(y_test, y_pred)

print(f"Train AUC: {train_auc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print(f"Optimal threshold: {optimal_threshold:.3f}")
print(f"Recall at optimal threshold: {recall:.3f}")

In [ ]:
from sklearn.metrics import roc_curve, recall_score, roc_auc_score
from sklearn.preprocessing import SplineTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

def create_splines(data_train, data_test, feature, n_knots, degree):
    spline = SplineTransformer(n_knots=n_knots, degree=degree)
    
    X_train_spline = spline.fit_transform(data_train[feature].values.reshape(-1, 1))
    X_test_spline = spline.transform(data_test[feature].values.reshape(-1, 1))
    
    return X_train_spline, X_test_spline

features = ['feature_1', 'feature_2', 'feature_3', 'feature_4']
feature_pairs = [(i, j) for i in features for j in features if i < j]

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'spline_n_knots': [3, 5, 7],
    'spline_degree': [2, 3, 4]
}

best_score = -np.inf
best_params = None
best_model = None
best_X_train = None
best_X_test = None

for n_knots in param_grid['spline_n_knots']:
    for degree in param_grid['spline_degree']:
        X_train_splines = []
        X_test_splines = []
        
        for feature in features:
            X_train_spline, X_test_spline = create_splines(train, test, feature, n_knots, degree)
            X_train_splines.append(X_train_spline)
            X_test_splines.append(X_test_spline)

        X_train_interactions = []
        X_test_interactions = []

        for pair in feature_pairs:
            interaction_train = train[pair[0]] * train[pair[1]]
            interaction_test = test[pair[0]] * test[pair[1]]
            X_train_interactions.append(interaction_train.values.reshape(-1, 1))
            X_test_interactions.append(interaction_test.values.reshape(-1, 1))

        X_train_final = np.hstack(X_train_splines + X_train_interactions)
        X_test_final = np.hstack(X_test_splines + X_test_interactions)

        base_model = LogisticRegression(solver='liblinear')
        grid_search = GridSearchCV(
            base_model,
            {'C': param_grid['C'], 'penalty': param_grid['penalty']},
            cv=5,
            scoring='roc_auc'
        )
        grid_search.fit(X_train_final, train['y'])
        
        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_params = {
                'C': grid_search.best_params_['C'],
                'penalty': grid_search.best_params_['penalty'],
                'spline_n_knots': n_knots,
                'spline_degree': degree
            }
            best_model = grid_search.best_estimator_
            best_X_train = X_train_final
            best_X_test = X_test_final

print("\nBest Parameters:")
print(best_params)

feature_names = []
for feature in features:
    feature_names.extend([f"{feature}_spline_{i}" for i in range(best_params['spline_n_knots'] + best_params['spline_degree'] - 1)])
for pair in feature_pairs:
    feature_names.append(f"{pair[0]}_x_{pair[1]}")

coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': best_model.coef_[0]
})
print("\nModel Coefficients:")
print(coefficients)

test_pred_proba = best_model.predict_proba(best_X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, test_pred_proba)
optimal_threshold = thresholds[np.argmin(np.abs(fpr - (1-tpr)))]
y_pred = (test_pred_proba >= optimal_threshold).astype(int)

final_recall = recall_score(y_test, y_pred)
final_auc = roc_auc_score(y_test, test_pred_proba)

plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curve (Test Recall = {final_recall:.3f}, Test AUC = {final_auc:.3f})')
plt.show()

print(f"Optimal threshold: {optimal_threshold:.3f}")
print(f"Final test recall at optimal threshold: {final_recall:.3f}")
print(f"Final test AUC: {final_auc:.3f}")

print("\nModel Components:")
print("\nSplined Main Effects:")
for feature in features:
    print(f"- {feature}")

print("\nInteraction Terms:")
for pair in feature_pairs:
    print(f"- {pair[0]} × {pair[1]}")

In [ ]:
from sklearn.metrics import roc_curve, recall_score, roc_auc_score
from sklearn.preprocessing import SplineTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
from itertools import combinations, product

def create_splines(data_train, data_test, feature, n_knots=3, degree=2):
    spline = SplineTransformer(n_knots=n_knots, degree=degree)
    X_train_spline = spline.fit_transform(data_train[feature].values.reshape(-1, 1))
    X_test_spline = spline.transform(data_test[feature].values.reshape(-1, 1))
    return X_train_spline, X_test_spline

features = ['feature_1', 'feature_2', 'feature_3', 'feature_4']
feature_pairs = [(i, j) for i in features for j in features if i < j]

results = []
for r in range(len(features) + 1):
    for splined_features in combinations(features, r):
        for s in range(len(feature_pairs) + 1):
            for interaction_pairs in combinations(feature_pairs, s):
                X_train_components = []
                X_test_components = []
                
                feature_names = []
                
                for feature in features:
                    if feature in splined_features:
                        X_train_spline, X_test_spline = create_splines(train, test, feature)
                        X_train_components.append(X_train_spline)
                        X_test_components.append(X_test_spline)
                        feature_names.extend([f"{feature}_spline_{i}" for i in range(3 + 2 - 1)])
                    else:
                        X_train_components.append(train[feature].values.reshape(-1, 1))
                        X_test_components.append(test[feature].values.reshape(-1, 1))
                        feature_names.append(feature)

                for pair in interaction_pairs:
                    interaction_train = train[pair[0]] * train[pair[1]]
                    interaction_test = test[pair[0]] * test[pair[1]]
                    X_train_components.append(interaction_train.values.reshape(-1, 1))
                    X_test_components.append(interaction_test.values.reshape(-1, 1))
                    feature_names.append(f"{pair[0]}_x_{pair[1]}")

                X_train_final = np.hstack(X_train_components)
                X_test_final = np.hstack(X_test_components)

                model = LogisticRegression(C=10, penalty='l1', solver='liblinear')
                model.fit(X_train_final, train['y'])

                test_pred_proba = model.predict_proba(X_test_final)[:, 1]
                fpr, tpr, thresholds = roc_curve(y_test, test_pred_proba)
                optimal_threshold = thresholds[np.argmin(np.abs(fpr - (1-tpr)))]
                y_pred = (test_pred_proba >= optimal_threshold).astype(int)

                recall = recall_score(y_test, y_pred)
                auc = roc_auc_score(y_test, test_pred_proba)
                score = recall + auc

                results.append({
                    'splined_features': splined_features,
                    'interaction_terms': interaction_pairs,
                    'recall': recall,
                    'auc': auc,
                    'total_score': score
                })

results.sort(key=lambda x: x['total_score'], reverse=True)

print("\nResults sorted by total score (recall + AUC):")
for i, result in enumerate(results):
    print(f"\n{i+1}. Total Score: {result['total_score']:.3f}")
    print(f"Recall: {result['recall']:.3f}")
    print(f"AUC: {result['auc']:.3f}")
    print("Splined Features:", ', '.join(result['splined_features']) if result['splined_features'] else 'None')
    print("Interaction Terms:", ', '.join([f"({p[0]} × {p[1]})" for p in result['interaction_terms']]) if result['interaction_terms'] else 'None')

best_result = results[0]
print("\nBest Combination:")
print(f"Total Score: {best_result['total_score']:.3f}")
print(f"Recall: {best_result['recall']:.3f}")
print(f"AUC: {best_result['auc']:.3f}")
print("Splined Features:", ', '.join(best_result['splined_features']) if best_result['splined_features'] else 'None')
print("Interaction Terms:", ', '.join([f"({p[0]} × {p[1]})" for p in best_result['interaction_terms']]) if best_result['interaction_terms'] else 'None')